In [ ]:
# !pip install PyYAML

In [ ]:
import cv2
import glob
import numpy as np
import os
import yaml
from yaml.loader import SafeLoader
import matplotlib.pyplot as plt

In [ ]:
# load YAML
with open('datarefuge.yaml',mode='r') as f:
    data_yaml = yaml.load(f,Loader=SafeLoader)
    
labels = data_yaml['names']
print(labels)


In [ ]:
# load YOLO model
yolo = cv2.dnn.readNetFromONNX('C:/Users/debora.assis/Documents/GitHub/YOLOv5_bounding_box_from_masks/Predict/Model2/weights/yolov5_model.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [ ]:
def yolo_detect(path, save):
    count = []
    saida_dir = "saida_img/"



    # load the image
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_name = img_path.split("\\")[-1][:-4]
    print(img_name)
    image = img.copy()
    row, col, d = image.shape


    max_rc = max(row,col)
    input_image = np.zeros((max_rc,max_rc,3),dtype=np.uint8)
    input_image[0:row,0:col] = image
    
    
    INPUT_WH_YOLO = 640
    blob = cv2.dnn.blobFromImage(input_image,1/255,(INPUT_WH_YOLO,INPUT_WH_YOLO),swapRB=True,crop=False)
    yolo.setInput(blob)
    preds = yolo.forward() # detection or prediction from YOLO


    detections = preds[0]
    boxes = []
    confidences = []
    classes = []

    
    image_w, image_h = input_image.shape[:2]
    x_factor = image_w/INPUT_WH_YOLO
    y_factor = image_h/INPUT_WH_YOLO

    for i in range(len(detections)):
        row = detections[i]
        confidence = row[4] # confidence of detection 
        if confidence > 0.8:
            a = 0
            class_score = row[5:].max() 
            class_id = row[5:].argmax() 

            if class_score > 0.25:
                cx, cy, w, h = row[0:4]
                # construct bounding 
                left = int((cx - 0.5*w)*x_factor)
                top = int((cy - 0.5*h)*y_factor)
                width = int(w*x_factor)
                height = int(h*y_factor)

                box = np.array([left,top,width,height])

                # append values into the list
                confidences.append(confidence)
                boxes.append(box)
                classes.append(class_id)

    # clean
    boxes_np = np.array(boxes).tolist()
    confidences_np = np.array(confidences).tolist()
#     print(confidences_np)

    if len(boxes_np) > 0:
        # Encontre o índice da caixa com a maior confiança
        max_confidence_index = np.argmax(confidences_np)

        # Extraia a caixa delimitadora com a maior confiança
        x, y, w, h = boxes_np[max_confidence_index]
        bb_conf = int(confidences_np[max_confidence_index] * 100)
        class_id = classes[max_confidence_index]
        class_name = labels[class_id]

        text = f'{class_name}: {bb_conf}%'

        cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.rectangle(image, (x, y - 30), (x + w, y), (255, 170, 170), -1)
        cv2.putText(image, text, (x, y - 10), cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 0), 2)

        if save == True:
            if not os.path.exists(saida_dir):
                os.mkdir(saida_dir)

            roi_image = img[y:y + h, x:x + w]
            cv2.imwrite(saida_dir + img_name + '.jpg', cv2.cvtColor(roi_image, cv2.COLOR_BGR2RGB))

    else:
        print("None object found")
        a = 1
    plt.imshow(image)
    plt.show()
    return a


# Images 

In [ ]:
# path = 'D:/Glaucoma/Banco_de_dados/REFUGE/REFUGE-Test400/Test400/1_glaucoma'
path = 'C:/Users/debora.assis/Documents/YOLOv5_bounding_box_from_masks/PrepareData/data_test_refuge'
extensoes = ['jpg', 'jpeg', 'png']
imagens_path = [imagem for extensao in extensoes for imagem in glob.glob(f'{path}/*.{extensao}')]
i=0
for img_path in imagens_path:
    yolo_detect(img_path, save = False) # if save = True, save image of bounding box
    i = i+1
    

In [ ]:
# DRISHTI-GS
path = 'C:/Users/debora.assis/Documents/YOLOv5_bounding_box_from_masks/PrepareData/teste_externo'
extensoes = ['jpg', 'jpeg', 'png']
imagens_path = [imagem for extensao in extensoes for imagem in glob.glob(f'{path}/*.{extensao}')]
i= []
for img_path in imagens_path:
    a = yolo_detect(img_path, save = False) # if save = True, save image of bounding box
    i.append(a)
    

In [ ]:
sum(i)

In [ ]:
# ORIGA 
path = 'D:/Glaucoma/Banco_de_dados/ORIGA/glaucoma/'
extensoes = ['jpg', 'jpeg', 'png']
imagens_path = [imagem for extensao in extensoes for imagem in glob.glob(f'{path}/*.{extensao}')]
i=[]
for img_path in imagens_path:
    a = yolo_detect(img_path, save = False) # if save = True, save image of bounding box
    i.append(a)
    

In [ ]:
sum(i)

In [ ]:
# ORIGA 
path = 'D:/Glaucoma/Banco_de_dados/ORIGA/sanas/'
extensoes = ['jpg', 'jpeg', 'png']
imagens_path = [imagem for extensao in extensoes for imagem in glob.glob(f'{path}/*.{extensao}')]
ij= []
for img_path in imagens_path:
    a = yolo_detect(img_path, save = False) # if save = True, save image of bounding box
    ij.append(a)
    

In [ ]:
sum(ij)

In [ ]:
# HRF
path = 'D:/Glaucoma/Banco_de_dados/HRF/images/Glaucoma/'
extensoes = ['jpg', 'jpeg', 'png']
imagens_path = [imagem for extensao in extensoes for imagem in glob.glob(f'{path}/*.{extensao}')]
i=0
for img_path in imagens_path:
    yolo_detect(img_path, save = False) # if save = True, save image of bounding box
    i = i+1
    

In [ ]:
# HRF
path = 'D:/Glaucoma/Banco_de_dados/HRF/images/Normal/'
extensoes = ['jpg', 'jpeg', 'png']
imagens_path = [imagem for extensao in extensoes for imagem in glob.glob(f'{path}/*.{extensao}')]
i=[]
for img_path in imagens_path:
    a=yolo_detect(img_path, save = False) # if save = True, save image of bounding box
    i.append(a)
    

In [ ]:
# HRF
path = 'D:/Glaucoma/Banco_de_dados/HRF/images/Retinopatia/'

extensoes = ['jpg', 'jpeg', 'png']
imagens_path = [imagem for extensao in extensoes for imagem in glob.glob(f'{path}/*.{extensao}')]
i=[]
for img_path in imagens_path:
    a = yolo_detect(img_path, save = False) # if save = True, save image of bounding box
    i.append(a)
    

# Video

In [ ]:
cap = cv2.VideoCapture('video.mp4')
save = False
saida_dir = "saida_video"
while True:
    
    ret, frame = cap.read()
    if ret == False:
        print('Error to read video')
        break
        
    try:
        
        image = frame
        row, col, d = image.shape
        max_rc = max(row,col)
        input_image = np.zeros((max_rc,max_rc,3),dtype=np.uint8)
        input_image[0:row,0:col] = image

        INPUT_WH_YOLO = 640
        blob = cv2.dnn.blobFromImage(input_image,1/255,(INPUT_WH_YOLO,INPUT_WH_YOLO),swapRB=True,crop=False)
        yolo.setInput(blob)
        preds = yolo.forward() # detection or prediction from YOLO


        detections = preds[0]
        boxes = []
        confidences = []
        classes = []


        image_w, image_h = input_image.shape[:2]
        x_factor = image_w/INPUT_WH_YOLO
        y_factor = image_h/INPUT_WH_YOLO
        i = 0
        for i in range(len(detections)):
            row = detections[i]
            confidence = row[4] # confidence of detection 
            if confidence > 0.8:
                class_score = row[5:].max() 
                class_id = row[5:].argmax() 

                cx, cy, w, h = row[0:4]
                # construct bounding 
                left = int((cx - 0.5*w)*x_factor)
                top = int((cy - 0.5*h)*y_factor)
                width = int(w*x_factor)
                height = int(h*y_factor)

                box = np.array([left,top,width,height])

                # append values into the list
                confidences.append(confidence)
                boxes.append(box)
                classes.append(class_id)

        # clean
        boxes_np = np.array(boxes).tolist()
        confidences_np = np.array(confidences).tolist()
    #     print(confidences_np)

#         if len(boxes_np) > 0:
            # Encontre o índice da caixa com a maior confiança
        max_confidence_index = np.argmax(confidences_np)

        # Extraia a caixa delimitadora com a maior confiança
        x, y, w, h = boxes_np[max_confidence_index]
        bb_conf = np.round(confidences_np[max_confidence_index],2)
        class_id = classes[max_confidence_index]
        class_name = labels[class_id]

        text = f'Conf: {bb_conf}'
        
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255,0), 2)
#             cv2.rectangle(image, (x, y - 30), (x + w, y), (255, 170, 170), -1)
        cv2.putText(image, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255,0), 2)
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.show()
        cv2.imwrite('image_' + str(i) + '.jpg', image)
        i = i + 1
    except:
        pass
    
    
    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()
cap.release()
